In [ ]:
from langchain_community.utilities import SQLDatabase


from langchain_core.prompts import ChatPromptTemplate


c:\Users\ashir\OneDrive\Desktop\text2sql\Text-to-SQL-Chatbot\text2sql\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
! pip install langchain_google_genai

In [5]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI

In [6]:
! pip install pymysql

In [7]:
#connect to db
host = "localhost"
port = "3306"
username = "root"
password = "arushisql"
database_schema = "text_to_sql"

mysql_uri = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database_schema}"

db = SQLDatabase.from_uri(mysql_uri, sample_rows_in_table_info= 2)

In [9]:
db = SQLDatabase.from_uri(mysql_uri, sample_rows_in_table_info= 1)

context = db.get_table_info()

context

'\nCREATE TABLE `2017_budgets` (\n\t`Product Name` TEXT, \n\t`2017 Budgets` DOUBLE\n)COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4\n\n/*\n1 rows from 2017_budgets table:\nProduct Name\t2017 Budgets\nProduct 1\t3016489.2089999998\n*/\n\n\nCREATE TABLE customers (\n\t`Customer Index` INTEGER, \n\t`Customer Names` TEXT\n)COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4\n\n/*\n1 rows from customers table:\nCustomer Index\tCustomer Names\n1\tGeiss Company\n*/\n\n\nCREATE TABLE products (\n\t`Index` INTEGER, \n\t`Product Name` TEXT\n)COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4\n\n/*\n1 rows from products table:\nIndex\tProduct Name\n1\tProduct 1\n*/\n\n\nCREATE TABLE regions (\n\tid INTEGER, \n\tname TEXT, \n\tcounty TEXT, \n\tstate_code TEXT, \n\tstate TEXT, \n\ttype TEXT, \n\tlatitude DOUBLE, \n\tlongitude DOUBLE, \n\tarea_code INTEGER, \n\tpopulation INTEGER, \n\thouseholds INTEGER, \n\tmedian_income INTEGER, \n\tland_area INTEGER, \

In [10]:
#create LLM prompt template
from langchain_core.prompts import ChatPromptTemplate

template = """Based on the table schema below, write an SQL query that would answer the user's question:
Remember : Only provide the sql query, don't include anything else. Provide me sql query in a 
single line don't add line breaks
Table Schema: {schema}
Question : {question}
SQL Query:
 """

prompt = ChatPromptTemplate.from_template(template)

In [11]:
#get the schema of DB
def get_schema(db):
    schema = db.get_table_info()
    return schema

In [ ]:
llm = ChatGoogleGenerativeAI(
    model = "gemini-2.0-flash",
    api_key = "ABC"
)

In [13]:
# create sql query chain using llm and the prompt template

sql_chain = (
    RunnablePassthrough.assign(schema = lambda _:get_schema(db))
    | prompt
    | llm.bind(stop={"/nSQLResult:"})
    | StrOutputParser()
)

In [14]:
# test sql query chain with sample q

resp = sql_chain.invoke({"question": "What is the total 'Line Total' for Geiss Company"})
print(resp)

```sql
SELECT sum(T1.`Line Total`) FROM sales_order AS T1 INNER JOIN customers AS T2 ON T1.`Customer Name Index` = T2.`Customer Index` WHERE T2.`Customer Names` = 'Geiss Company'
```


In [15]:
# test sql query chain with sample q

resp = sql_chain.invoke({"question": "What was the budget of Product 12"})
print(resp)

```sql
SELECT `2017 Budgets` FROM `2017_budgets` WHERE `Product Name` = 'Product 12'
```


In [14]:
resp = """```sql
SELECT `2017 Budgets` FROM `2017_budgets` WHERE `Product Name` = 'Product 12'
```"""

In [16]:
import re

query = re.search(r"```sql\s*(.*?)\s*```", resp, re.DOTALL | re.IGNORECASE)

if query:
    query = query.group(1).strip()

In [17]:
db.run(query)

'[(1356976.996,)]'

### RAGAS Implementions

In [17]:

! pip install ragas

In [18]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

In [ ]:
# from Langchain_openai import ChatOpenAI
# from langchain_embeddings import OpenAIEmbeddings

In [19]:
! pip install langchain-groq

In [ ]:
from langchain_groq import ChatGroq

llm = ChatGoogleGenerativeAI(
    model = "gemini-2.0-flash",
    api_key = "ABC"
)

In [21]:
! pip install transformers
! pip install torch
! pip install sentence-transformers

In [22]:
# from langchain.embeddings import HuggingFaceEmbeddings

# embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformer/all-mpnet-base-v2")

from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-mpnet-base-v2")

In [23]:
# evaluator_llm = LangchainLLMWrapper(llm)
# evaluator_embeddings = LangchainEmbeddingsWrapper(embeddings)

embeddings = LangchainEmbeddingsWrapper(model)

C:\Users\ashir\AppData\Local\Temp\ipykernel_17016\3148150208.py:4: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  embeddings = LangchainEmbeddingsWrapper(model)


In [24]:
evaluator_llm = LangchainLLMWrapper(llm)

C:\Users\ashir\AppData\Local\Temp\ipykernel_17016\2998763756.py:1: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use the modern LLM providers instead: from ragas.llms.base import llm_factory; llm = llm_factory('gpt-4o-mini') or from ragas.llms.base import instructor_llm_factory; llm = instructor_llm_factory('openai', client=openai_client)
  evaluator_llm = LangchainLLMWrapper(llm)


In [25]:
from ragas import evaluate
from ragas.metrics import AspectCritic, RubricsScore

maliciousness_definition = (
    "Is the response intended to harm, deceive, or exploit users?"
)


aspect_critic = AspectCritic(
    name = "maliciousness",
    definition = maliciousness_definition,
    llm = evaluator_llm
)


# adopt Google's helpfulness_prompt_template

helpfulness_rubrics = {
    "score1_description": "Response is useless/irrelevant, contains inaccurate/deceptive/misleading information and/or contains harmful/offensive content. The user would feel not at all satisfied with the content in the response.",
    "score2_description": "Response is minimally relevant to the instruction and may provide some vaguely useful information, but it lacks clarity and detail. It might contain minor inaccuracies. The user would feel only slightly satisfied with the content in the response.",
    "score3_description": "Response is relevant to the instruction and provides some useful content, but could be more relevant, well-defined, comprehensive, and/or detailed. The user would feel somewhat satisfied with the content in the response.",
    "score4_description": "Response is very relevant to the instruction, providing clearly defined information that addresses the instruction's core needs.  It may include additional insights that go slightly beyond the immediate instruction.  The user would feel quite satisfied with the content in the response.",
    "score5_description": "Response is useful and very comprehensive with well-defined key details to address the needs in the instruction and usually beyond what explicitly asked. The user would feel very satisfied with the content in the response.",
}

rubrics_score = RubricsScore(name="helpfulness", rubrics=helpfulness_rubrics, llm=evaluator_llm) 


In [26]:
from ragas import evaluate
from ragas.metrics import ContextPrecision, Faithfulness

context_precision = ContextPrecision(llm=evaluator_llm)
faithfulness = Faithfulness(llm=evaluator_llm)

In [27]:
retrieved_contexts = [context]

In [28]:
import re

user_inputs = [
    "What was the budget of Product 12",
    "What are the names of all products in the products table?",
    "List all customer names from the customers table.",
    "Find the name and state of all regions in the regions table.",
    "What is the name of the customer with Customer Index = 1"
]

responses = []

for question in user_inputs:
    resp = sql_chain.invoke({"question": question})
    match = re.search(r"```sql\s*(.*?)\s*```", resp, re.DOTALL | re.IGNORECASE)
    if match:
        query = match.group(1).strip()
        responses.append(query)

In [29]:
references=["SELECT `2017 Budgets` FROM `2017_budgets` WHERE `Product Name` = 'Product 12';",
            "SELECT `Product Name`ROM products;",
            "SELECT `Customer Names`FROM customers;",
            "SELECT name, state FROM regions;",
            "SELECT `Customer Names` FROM customers WHERE `Customer Index` = 1;"]

In [30]:
from ragas.dataset_schema import SingleTurnSample, EvaluationDataset
n = len(user_inputs)
samples = []



In [31]:
for i in range(n):

    sample = SingleTurnSample(
        user_input=user_inputs[i],
        retrieved_contexts=list(retrieved_contexts),
        response=responses[i],
        reference=references[i],
    )
    samples.append(sample)

In [32]:
ragas_eval_dataset = EvaluationDataset(samples=samples)
ragas_eval_dataset.to_pandas()

,user_input,retrieved_contexts,response,reference
0,What was the budget of Product 12,[\nCREATE TABLE `2017_budgets` (\n\t`Product N...,SELECT `2017 Budgets` FROM `2017_budgets` WHER...,SELECT `2017 Budgets` FROM `2017_budgets` WHER...
1,What are the names of all products in the prod...,[\nCREATE TABLE `2017_budgets` (\n\t`Product N...,SELECT `Product Name` FROM products,SELECT `Product Name`ROM products;
2,List all customer names from the customers table.,[\nCREATE TABLE `2017_budgets` (\n\t`Product N...,SELECT `Customer Names` FROM customers,SELECT `Customer Names`FROM customers;
3,Find the name and state of all regions in the ...,[\nCREATE TABLE `2017_budgets` (\n\t`Product N...,"SELECT name, state FROM regions","SELECT name, state FROM regions;"
4,What is the name of the customer with Customer...,[\nCREATE TABLE `2017_budgets` (\n\t`Product N...,SELECT `Customer Names` FROM Customers WHERE `...,SELECT `Customer Names` FROM customers WHERE `...


In [33]:
from ragas import evaluate

ragas_metrics = [ context_precision, rubrics_score]

result = evaluate(
    metrics=ragas_metrics,
    dataset=ragas_eval_dataset
)
result

Evaluating: 100%|██████████| 10/10 [00:40<00:00,  4.02s/it]


{'context_precision': 1.0000, 'helpfulness': 5.0000}